In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install bs4
!pip install seqeval
!pip install conllu
!pip install vncorenlp

In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip install .

In [ ]:
!cp /kaggle/input/ner-dataset/ner_dev.txt ./dev.txt
!cp /kaggle/input/ner-dataset/ner_dev.txt ./test.txt
!cp /kaggle/input/ner-dataset/ner_train.txt ./train.txt

In [ ]:
%cd /kaggle/working
!git clone https://github.com/vncorenlp/VnCoreNLP
%cd /kaggle/working/transformers/examples/legacy/token-classification



In [ ]:
from vncorenlp import VnCoreNLP

annotator = VnCoreNLP("/kaggle/working/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 


In [ ]:
%%writefile labels.txt
B-team
I-team
O

In [ ]:
TEAM_NAMES = [
    ["Barcelona", "Barca", "BARCELONA"],
    ["Man United", "M.U", "MU", "Man Utd", "United", "Manchester United"],
    ["B.Bình Dương", "Bình Dương"],
    ["AC Milan", "Milan"],
    ["Sociedad", "Real Sociedad"], 
    ["SHB.Đà Nẵng", "SHB Đà Nẵng"],
    ["Than Quảng Ninh", "Than.QN"],
    ["Bayern Munich", "Bayern"],
    ["Bayer Leverkusen", "Leverkusen"],
    ["Inter Milan", "Inter"],
    ["Real Madrid", "Real", "REAL MADRID"],
    ["Atletico Madrid", "Atletico"],
    ["Stoke City", "Stoke"],
    ["LA GALAXY", "LA Galaxy"],
    ["Manchester City", "Man City"],
    ["U15 SLNA", "SLNA"],
    ["U15 HAGL", "HAGL"],
    ["Mokpo City FC", "CLB Mokpo"],
    ["Athletic Bilbao", "Athletic"],
    ["U22 VN", "U22 Việt Nam", "Việt Nam", "U22 ViệtNam"],
    ["U.22 Lào", "U22 Lào"],
    ["U22 Campuchia", "Campuchia"],
    ["Futsal Thái Lan", "Thái Lan", "futsal Thái Lan"],
    ["U23 Manchester City", "U23 Man City"],
    ["U23 Arsenal", "Arsenal"],
    ["U22 Thái Lan", "Thái Lan", "U 22 Thái Lan"],
    ["Tuyển nữ Việt Nam", "đội tuyển Việt Nam", "đội tuyển nữ Việt Nam", "tuyển nữ Việt Nam", "tuyển Việt Nam", "ĐT nữ Việt Nam", "nữ Việt Nam", "VN", "Việt Nam", "bóng đá nữ Việt Nam"],
    ["nữ futsal Việt Nam", "futsal nữ Việt Nam", "Việt Nam"],
    ["Burton Albion", "Burton"],
    ["Chelsea", "The Blues"],
    ["FLC Thanh Hóa", "FLC THANH HÓA"],
    ["Olympic Bahrain", "Bahrain"],
    ["RB Leipzig", "Leipzig"],
]


def my_concatenate(str1, str2):
    if (not str1.endswith('.')) and (not str2.startswith('.')) and (str1!='') :
        return str1 + ' . ' + str2
    return str1 + ' ' +str2

def create_context_from_mi_mr(json_obj, match_info = True, match_result = True):
    result  = {'match_info': [], 'match_result': []}
    for html in json_obj['html_annotation']:
        soup = BeautifulSoup(html)

        if match_info:
            match_info_ls = soup.find_all('span', {'class':'match_info'})
            for i in match_info_ls:
                result['match_info'].append(i.text)
        if match_result:
            match_result_ls = soup.find_all('span', {'class':'match_result'})
            for i in match_result_ls:
                result['match_result'].append(i.text)
    return result

def get_match_summary_as_text(json_obj, label = 'players'):
    team1  = json_obj['match_summary']['players']['team1']
    team2 = json_obj['match_summary']['players']['team2']
    return team1, team2
def get_team_name_alias(team_name):
    for ls in TEAM_NAMES:
        if team_name in ls:
            return ls
    return [team_name]
def team_name_appear_in_text_filter(text, team_name_ls):
    result = []
    #full_text = ' '.join([i for i in text_list])
    #print('full_text',full_text)
    for name in team_name_ls:
        if name in text:
            result.append(name)
    return result
def nomalize1(text):
    result = text.strip('\n').strip(' ').strip('.')
    if not result.endswith('.'):
        result += ' .'
    return result

def do_segmentation(text):
    #return annotator.tokenize(text)
    result = ' '.join([' '.join(i) for i in annotator.tokenize(text)])
    #result = nomalize1(result)
    return result


# 
def create_team_ner_label(text, name_list):
    text_split = text.split()
    ner_list = ['O']*len(text_split)

    for name in name_list:
        if name in text:
            name_split = name.split()
            start_index = text_split.index(name_split[0])
            if len(name_split) == 1:
                ner_list[start_index] = 'B-team'     
            else:
                end_index = start_index + len(name_split) - 1
                for i in range(start_index, end_index + 1):
                    if i==end_index:
                        ner_list[i] = 'I-team'
                    else:
                        ner_list[i] = 'B-team'                    
    return (text_split, ner_list)



# util
def find_sub_list(sub_ls, ls):
    sub_ls_len = len(sub_ls)
    result_ls = []
    for i in [i for i, v in enumerate(ls) if v==sub_ls[0]]:
        if ls[i:i+sub_ls_len] == sub_ls:
            result_ls.append((i,i+sub_ls_len-1))
    return result_ls


def ibo_annotate(ls, begin, end, ent = 'team'):
    if begin == end:
        ls[begin]='B-' + ent
    else:
        ls[begin]='B-' + ent
        for i in range(begin+1,end+1):
                ls[i] = 'I-' + ent

def create_team_ner_label2(text, name_list):
    text_split = text.split()
    ner_list = ['O']*len(text_split)
    
    for name in name_list:
        name = name.split()
        #print(f'sub ls:{name}')
        ids = find_sub_list(name, text_split)
        #print(ids)
        for item in ids:
            begin, end = item
            if ner_list[begin] == 'O':
                ibo_annotate(ner_list, begin, end, 'team')
    return text_split, ner_list
            
            

In [ ]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning the library models for named entity recognition on CoNLL-2003. """
import logging
import os
import sys
from dataclasses import dataclass, field
from importlib import import_module
from typing import Dict, List, Optional, Tuple

import numpy as np
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch import nn

import transformers
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed)
from transformers.trainer_utils import is_main_process
from utils_ner import Split, TokenClassificationDataset, TokenClassificationTask


logger = logging.getLogger(__name__)
SEED = 42
DO_PREDICT = True
OUTPUT_DIR = './'
DO_TRAIN = False
LOCAL_RANK = -1
USE_FAST = False
FP16 = False
LABELS = 'labels.txt'
DEVICE = 'cuda'
OVERWRITE_OUTPUT_DIR = False
MODEL_NAME_OR_PATH = '/kaggle/input/pretrain-model-news-summ/ner_team_pretrained_model'
MAX_SEQ_LENGTH = 128
TASK_TYPE = 'NER'
OVERWRITE_CACHE= True
CACHE_DIR = None
DATA_DIR = './'
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    task_type: Optional[str] = field(
        default="NER", metadata={"help": "Task type to fine tune in training (e.g. NER, POS, etc)"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    data_dir: str = field(
        metadata={"help": "The input data dir. Should contain the .txt files for a CoNLL-2003-formatted task."}
    )
    labels: Optional[str] = field(
        default=None,
        metadata={"help": "Path to a file containing all labels. If not specified, CoNLL-2003 labels are used."},
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )


        
        

if (
    os.path.exists(OUTPUT_DIR)
    and os.listdir(OUTPUT_DIR)
    and DO_TRAIN
    and not OVERWRITE_OUTPUT_DIR
):
    raise ValueError(
        f"Output directory ({OUTPUT_DIR}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

module = import_module("tasks")
try:
    token_classification_task_clazz = getattr(module, TASK_TYPE)
    token_classification_task: TokenClassificationTask = token_classification_task_clazz()
except AttributeError:
    raise ValueError(
        f"Task {TASK_TYPE} needs to be defined as a TokenClassificationTask subclass in {module}. "
        f"Available tasks classes are: {TokenClassificationTask.__subclasses__()}"
    )

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if LOCAL_RANK in [-1, 0] else logging.WARN,
)

# Set the verbosity to info of the Transformers logger (on main process only):
if is_main_process(LOCAL_RANK):
    transformers.utils.logging.set_verbosity_info()
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()
#logger.info("Training/evaluation parameters %s", training_args)

# Set seed
set_seed(SEED)

# Prepare CONLL-2003 task
labels = token_classification_task.get_labels(LABELS)
label_map: Dict[int, str] = {i: label for i, label in enumerate(labels)}
num_labels = len(labels)

# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

config = AutoConfig.from_pretrained(
    MODEL_NAME_OR_PATH,
    num_labels=num_labels,
    id2label=label_map,
    label2id={label: i for i, label in enumerate(labels)},
    cache_dir=CACHE_DIR,
)
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME_OR_PATH,
    cache_dir=CACHE_DIR,
    use_fast=USE_FAST,
)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME_OR_PATH,
    from_tf=bool(".ckpt" in MODEL_NAME_OR_PATH),
    config=config,
    cache_dir=CACHE_DIR,
)

# Get datasets


def align_predictions(predictions: np.ndarray, label_ids: np.ndarray) -> Tuple[List[int], List[int]]:
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                out_label_list[i].append(label_map[label_ids[i][j]])
                preds_list[i].append(label_map[preds[i][j]])

    return preds_list, out_label_list

def compute_metrics(p: EvalPrediction) -> Dict:
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

# Data collator
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8) if FP16 else None

# Initialize our Trainer
trainer = Trainer(
    model=model,
    #args=training_args,
    #train_dataset=train_dataset,
    #eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)


# # Predict
# if DO_PREDICT:
#     test_dataset = TokenClassificationDataset(
#         token_classification_task=token_classification_task,
#         data_dir=DATA_DIR,
#         tokenizer=tokenizer,
#         labels=labels,
#         model_type=config.model_type,
#         max_seq_length=MAX_SEQ_LENGTH,
#         overwrite_cache=OVERWRITE_CACHE,
#         mode=Split.test,
#     )

#     predictions, label_ids, metrics = trainer.predict(test_dataset)
#     preds_list, _ = align_predictions(predictions, label_ids)

#     output_test_results_file = os.path.join(OUTPUT_DIR, "test_results.txt")
#     if trainer.is_world_process_zero():
#         with open(output_test_results_file, "w") as writer:
#             for key, value in metrics.items():
#                 logger.info("  %s = %s", key, value)
#                 writer.write("%s = %s\n" % (key, value))

#     # Save predictions
#     output_test_predictions_file = os.path.join(OUTPUT_DIR, "test_predictions.txt")
#     if trainer.is_world_process_zero():
#         with open(output_test_predictions_file, "w") as writer:
#             with open(os.path.join(DATA_DIR, "test.txt"), "r") as f:
#                 token_classification_task.write_predictions_to_file(writer, f, preds_list)



def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()


# if __name__ == "__main__":
#     main()
    

    
def predict():
    # Predict
        test_dataset = TokenClassificationDataset(
            token_classification_task=token_classification_task,
            data_dir=DATA_DIR,
            tokenizer=tokenizer,
            labels=labels,
            model_type=config.model_type,
            max_seq_length=MAX_SEQ_LENGTH,
            overwrite_cache=OVERWRITE_CACHE,
            mode=Split.test,
        )

        predictions, label_ids, metrics = trainer.predict(test_dataset)
        preds_list, _ = align_predictions(predictions, label_ids)

        output_test_results_file = os.path.join(OUTPUT_DIR, "test_results.txt")
        if trainer.is_world_process_zero():
            with open(output_test_results_file, "w") as writer:
                for key, value in metrics.items():
                    logger.info("  %s = %s", key, value)
                    writer.write("%s = %s\n" % (key, value))

        # Save predictions
        output_test_predictions_file = os.path.join(OUTPUT_DIR, "test_predictions.txt")
        if trainer.is_world_process_zero():
            with open(output_test_predictions_file, "w") as writer:
                with open(os.path.join(DATA_DIR, "test.txt"), "r") as f:
                    token_classification_task.write_predictions_to_file(writer, f, preds_list)
                    
def predict2():
    # Predict
        test_dataset = TokenClassificationDataset(
            token_classification_task=token_classification_task,
            data_dir=DATA_DIR,
            tokenizer=tokenizer,
            labels=labels,
            model_type=config.model_type,
            max_seq_length=MAX_SEQ_LENGTH,
            overwrite_cache=OVERWRITE_CACHE,
            mode=Split.test,
        )

        predictions, label_ids, metrics = trainer.predict(test_dataset)
        preds_list, _ = align_predictions(predictions, label_ids)
#         print(preds_list)
#         for i in preds_list:
#             print(len(i))
        output_test_results_file = os.path.join(OUTPUT_DIR, "team_test_results.txt")
        if trainer.is_world_process_zero():
            print('is_world_process_zero')
            with open(output_test_results_file, "w") as writer:
                for key, value in metrics.items():
                    logger.info("  %s = %s", key, value)
                    writer.write("%s = %s\n" % (key, value))

        # Save predictions
        output_test_predictions_file = os.path.join(OUTPUT_DIR, "team_test_predictions.txt")
        if trainer.is_world_process_zero():
            with open(output_test_predictions_file, "w") as writer:
                with open(os.path.join(DATA_DIR, "test.txt"), "r") as f:
                    token_classification_task.write_predictions_to_file(writer, f, preds_list)

In [ ]:
# only 1 sentence
def extract_ne_from_ner_prediction(txt_predict_file, ne = 'team'):
    with open(txt_predict_file, 'r') as f:
        results= []
        ls = list(map(lambda x:tuple(x.strip('\n').split()), f))
        #print(ls,'\n')
        i=0
        while True:
            if i>=len(ls): break
            w, p = ls[i]
            if p=='B-'+ne:
                name = ''
                name += w
                j = i + 1
                while True:
                    if j>=len(ls): break
                    if ls[j][1] == ('I-'+ ne):
                        name += ' ' + ls[j][0]
                    else: break
                    j+=1
                #print(f'\"{name}\"')
                results.append(name)
            i+=1
    return results
            
                
#extract_ne_from_ner_prediction('team_test_predictions.txt', 'team')

In [ ]:
## cell này chạy thử tìm 2 đội (giả sử bước classsify đúng match_info và match_result)

import json
from bs4 import BeautifulSoup
import pandas as pd
# predict team

train_path = '/kaggle/input/zacdata/train/train.jsonl'
with open('label_predict.txt', 'w') as f_pred:
    with open(train_path,'r') as f:

        ls = list(f)
        for item in ls[600:650]: # đã train trên 0:600 bài, bên chạy thử từ 600: cuối
            team_predict_ls = []


            json_obj = json.loads(item)
            train_id = json_obj['train_id']
            # create context from match_info, match_result
            context_dict = create_context_from_mi_mr(json_obj)
            # get team label (as text)
            team1, team2 = get_match_summary_as_text(json_obj, label = 'players')
            # get team name alias
            team1_ls = get_team_name_alias(team1)
            team2_ls = get_team_name_alias(team2)
            #print(team1_ls, '-' ,team2_ls)
            #print(context_dict)
            #print('')
            full_text = ' '.join(context_dict['match_info']) + ' ' + ' '.join(context_dict['match_result'])
            #print(full_text)
            team1_ls = team_name_appear_in_text_filter(full_text, team1_ls)
            team2_ls = team_name_appear_in_text_filter(full_text, team2_ls)
            #print(context_dict)
            team1_ls = [do_segmentation(i) for i in team1_ls]
            team2_ls = [do_segmentation(i) for i in team2_ls]

            print('TEAM NAME LABEL:',team1_ls, '-' ,team2_ls)
            #ner_data = []
            #print(context_dict)
            for field in ['match_info', 'match_result']:
                for item in context_dict[field]:
                    item_segmented = do_segmentation(item)
                    #print(item_segmented)

                    text_split, ner_list = create_team_ner_label2(item_segmented, team1_ls + team2_ls)
                    #print(text_split, [-1]*len(ner_list))
                    with open('test.txt', 'w') as f_ner:
                        #print('heeeee')
                        for t in  text_split:
                            #print(t)
                            f_ner.write(f'{t} O\n')
                    #print(pd.read_csv('test.txt', sep = ' '))
                    predict2()
                    #print(pd.read_csv('team_test_predictions.txt', sep = ' '))
                    #print('--------------')
                    team_predict = extract_ne_from_ner_prediction('team_test_predictions.txt', 'team')
                    team_predict_ls.extend(team_predict)
                    f_pred.write(f'train_id: {train_id} groundtruth:{team1_ls}-{team2_ls} predict:{team_predict_ls}\n')
                    print('team predict:',team_predict_ls)
                    print('--------------')
            print('=================================')

In [ ]:
!cat label_predict.txt